In [1]:
import os
import torch
import time
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [18]:
# Створити папку для зображень
os.makedirs("progan_samples_images", exist_ok=True)
os.makedirs("progan_samples_models", exist_ok=True)

# Пристрій
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Параметри
z_dim = 128
batch_size = 128
learning_rate = 2e-4
total_epochs = 300  # загальна кількість епох
fade_epochs = 100    # епох на плавний перехід
start_step = 0        # почати з 4x4
num_steps = 3         # максимум 28x28
img_channels = 128
fixed_noise = torch.randn(16, z_dim, device=device)
gif_path = 'progan_progress_v2.gif'

In [19]:
# Датасет MNIST
transform = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # нормалізація в [-1, 1]
])

dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)


In [21]:
# Генератор
class Generator(nn.Module):
    def __init__(self, z_dim, img_channels):
        super().__init__()
        self.z_dim = z_dim

        self.initial_conv = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 128, 4, 1, 0),  # з (z_dim, 1x1) в (128, 4x4)
            nn.LeakyReLU(0.2)
        )

        self.progression = nn.ModuleList([
            self._block(128, 64),    # 8x8
            self._block(64, 32),     # 16x16
            self._block(32, img_channels, final=True)  # 28x28
        ])

    def _block(self, in_channels, out_channels, final=False):
        if final:
            return nn.Sequential(
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_channels, out_channels, 3, padding=1),
                nn.Tanh()
            )
        else:
            return nn.Sequential(
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_channels, out_channels, 3, padding=1),
                nn.LeakyReLU(0.2)
            )

    def forward(self, x, step, alpha):
        out = x.view(x.shape[0], self.z_dim, 1, 1)
        out = self.initial_conv(out)

        for i in range(step):
            out = self.progression[i](out)

        if step < len(self.progression):
            skip = nn.functional.interpolate(out, scale_factor=2)
            out = self.progression[step](skip)

        return out

In [23]:
# Дискримінатор
class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super().__init__()

        self.progression = nn.ModuleList([
            self._block(img_channels, 32),   # 28x28 -> 14x14
            self._block(32, 64),              # 14x14 -> 7x7
            self._block(64, 128),             # 7x7 -> 4x4
        ])

        self.final_conv = nn.Sequential(
            nn.Conv2d(128, 1, 4, 1, 0),  # з 4x4 в 1x1
        )

    def _block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=2, padding=1),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x, step, alpha):
        for i in range(step):
            x = self.progression[i](x)

        if step < len(self.progression):
            x = self.progression[step](x)

        x = self.final_conv(x)
        return x.view(x.shape[0], -1)

In [24]:
# Ініціалізація
gen = Generator(z_dim, img_channels).to(device)
disc = Discriminator(img_channels).to(device)

optimizer_G = optim.Adam(gen.parameters(), lr=learning_rate, betas=(0.3, 0.99))
optimizer_D = optim.Adam(disc.parameters(), lr=learning_rate, betas=(0.3, 0.99))

criterion = nn.BCEWithLogitsLoss()

In [25]:
# Навчання
step = start_step
alpha = 1.0
start = time.time()

for epoch in range(total_epochs):
    for real_imgs, _ in dataloader:
        real_imgs = real_imgs.to(device)
        print(real_imgs.shape)
        # Тренування дискримінатора
        noise = torch.randn(batch_size, z_dim, device=device)
        fake_imgs = gen(noise, step, alpha).detach()
        
        real_preds = disc(real_imgs, step, alpha)
        fake_preds = disc(fake_imgs, step, alpha)

        real_labels = torch.ones_like(real_preds)
        fake_labels = torch.zeros_like(fake_preds)

        d_loss_real = criterion(real_preds, real_labels)
        d_loss_fake = criterion(fake_preds, fake_labels)
        d_loss = (d_loss_real + d_loss_fake) / 2

        optimizer_D.zero_grad()
        d_loss.backward()
        optimizer_D.step()

        # Тренування генератора
        noise = torch.randn(batch_size, z_dim, device=device)
        fake_imgs = gen(noise, step, alpha)
        fake_preds = disc(fake_imgs, step, alpha)

        g_loss = criterion(fake_preds, real_labels)

        optimizer_G.zero_grad()
        g_loss.backward()
        optimizer_G.step()
    
    # Оновлення alpha для плавного переходу
    alpha += 1.0 / fade_epochs
    alpha = min(alpha, 1.0)

    # Збереження результатів
    if epoch % 10 == 0:
        gen.eval()
        with torch.no_grad():
            fake_test = gen(fixed_noise, step, alpha)
            fake_test = (fake_test + 1) / 2  # відновлення [0, 1] для збереження
            save_image(fake_test, f"progan_samples_images/{epoch:05d}.png", nrow=4)
        gen.train()
        
        torch.save(gen.state_dict(), f"progan_samples_models/generator_epoch_{epoch}.pth")
        torch.save(disc.state_dict(), f"progan_samples_models/discriminator_epoch_{epoch}.pth")
       

    # Збільшення рівня step
    if (epoch + 1) % fade_epochs == 0 and step < num_steps:
        step += 1
        alpha = 0.0  # reset alpha для плавного переходу
        print(f"[Epoch {epoch}/{total_epochs}] Saved images and models.")

timer = time.time()-start
print ('Time for epochs {} is {} min'.format(num_epochs, int(timer/60)))

torch.Size([128, 1, 28, 28])


RuntimeError: Given groups=1, weight of size [32, 128, 3, 3], expected input[128, 1, 28, 28] to have 128 channels, but got 1 channels instead

In [ ]:
import os
import glob
import imageio
import numpy as np
import IPython.display as display

# Зчитуємо всі збережені картинки
imgs = sorted(
    glob.glob('progan_samples_images/*.png'),
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
)

frames = []

# Кількість проміжних кадрів між 2 основними
transition_frames = 5

for idx in range(len(imgs)-1):
    img1 = imageio.v3.imread(imgs[idx]).astype(np.float32) / 255.0
    img2 = imageio.v3.imread(imgs[idx+1]).astype(np.float32) / 255.0
    
    # Додаємо початковий кадр
    frames.append((img1 * 255).astype(np.uint8))
    
    # Плавні переходи між img1 і img2
    for t in range(1, transition_frames):
        alpha = t / transition_frames
        blended = (1 - alpha) * img1 + alpha * img2
        frames.append((blended * 255).astype(np.uint8))

# Додаємо останній кадр
last_img = imageio.v3.imread(imgs[-1])
frames.append(last_img)

# Зберігаємо фінальний плавний GIF
imageio.mimsave(gif_path, frames, duration=0.1)

# Показати в ноутбуці
display.Image(filename=gif_path)